#### Importações

In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
#from ctgan import CTGAN
import numpy as np

#### Carrega e pré processa os dados

In [5]:
# Column names for the dataset
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]

# Load the Adult dataset
train_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
test_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"

# Load training data
train_data = pd.read_csv(train_url, header=None, names=columns, na_values=["?"], skipinitialspace=True)

# Load test data
test_data = pd.read_csv(test_url, header=None, names=columns, na_values=["?"], skipinitialspace=True, skiprows=1)
test_data["income"] = test_data["income"].str.strip(".")  # Clean income column in test set

# Remove the 'fnlwgt' column
train_data = train_data.drop(columns=['fnlwgt', 'education-num'])
test_data = test_data.drop(columns=['fnlwgt', 'education-num'])

In [29]:
train_data['capital-gain'] = train_data['capital-gain'].astype(int)
test_data['capital-gain'] = test_data['capital-gain'].astype(int)

In [31]:
train_data = train_data[train_data['capital-gain'] != 99999]
test_data = test_data[test_data['capital-gain'] != 99999]

#### Geração de dados sintéticos

In [23]:
discrete_columns = [
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country',
    'income'
]
ctgan = CTGAN(epochs=1)
ctgan.fit(train_data, discrete_columns)

In [121]:
syndata = ctgan.sample(30000)

In [79]:
# Remove 1000 random samples from train_data
train_data = train_data.drop(train_data.sample(n=20000).index)

#### Transformações para modelagem

In [6]:
# Combine train and test data for consistent preprocessing
data = pd.concat([train_data, test_data], axis=0, ignore_index=True) #adicione e remova train_data e syndata para treinar ou nao com dados reais e sinteticos

# Combinar categorias da variável 'native-country'
data['native-country'] = data['native-country'].replace({
    country: 'Other' for country in data['native-country'].unique() if country != 'United-States'
})

# Encode the target variable
label_encoder = LabelEncoder()
data["income"] = label_encoder.fit_transform(data["income"])  # Encodes '<=50K' as 0 and '>50K' as 1


# Split features and target
X = data.drop(columns=["income"])
y = data["income"]

# Label Encoding para education, sex e race
for col in ["education", "sex", "native-country"]:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Define categorical and numerical columns
categorical_columns = X.select_dtypes(include=["object"]).columns
numerical_columns = X.select_dtypes(include=["int64", "float64"]).columns

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_columns),  # Scale numerical features
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_columns),  # One-hot encode categorical features
    ],
    remainder="passthrough"
)

# Split back into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=len(test_data), shuffle=False)

#### Pipeline para Regressão Logística

In [123]:
# Create a pipeline with preprocessing and classifier
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000)),
])

#### Pipeline para Random Forest

In [125]:
# Create a pipeline with preprocessing and Random Forest classifier
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)),
])

#### Treina o modelo

In [ ]:
# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))


#### Hyperparameter tunning Random Forest

In [ ]:
# Define the parameter grid
param_grid = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [None, 10, 20, 30, 40],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None],
    "bootstrap": [True, False],
}

# Create the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Use RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=50,  # Number of random combinations to try
    scoring="accuracy",
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all available processors
)

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Fit the model
random_search.fit(X_train_preprocessed, y_train)

# Best parameters
print("Best Parameters:", random_search.best_params_)

# Evaluate the best model
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_preprocessed)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

#### Treina XGBoost

In [10]:
import xgboost as xgb

In [25]:
train_dmatrix = xgb.DMatrix(data=X_train_preprocessed, label=y_train)

In [26]:
test_dmatrix = xgb.DMatrix(data=X_test_preprocessed, label=y_test)

In [47]:
model = xgb.train(
    params={"objective": "binary:logistic",  # Classificação binária
    "eval_metric": "auc",        # Métrica de avaliação
    "eta": 0.1,                      # Taxa de aprendizado (equivalente a learning_rate)
    "max_depth": 6,                   # Profundidade das árvores
    "nthread": -1},
    dtrain=train_dmatrix,
    num_boost_round=100,
    evals=[(test_dmatrix, "Test")]
)

[0]	Test-auc:0.88869
[1]	Test-auc:0.89441
[2]	Test-auc:0.90233
[3]	Test-auc:0.90325
[4]	Test-auc:0.90526
[5]	Test-auc:0.90534
[6]	Test-auc:0.90716
[7]	Test-auc:0.90772
[8]	Test-auc:0.90807
[9]	Test-auc:0.90868
[10]	Test-auc:0.90901
[11]	Test-auc:0.90956
[12]	Test-auc:0.91004
[13]	Test-auc:0.91041
[14]	Test-auc:0.91119
[15]	Test-auc:0.91184
[16]	Test-auc:0.91238
[17]	Test-auc:0.91295
[18]	Test-auc:0.91325
[19]	Test-auc:0.91369
[20]	Test-auc:0.91415
[21]	Test-auc:0.91436
[22]	Test-auc:0.91503
[23]	Test-auc:0.91538
[24]	Test-auc:0.91570
[25]	Test-auc:0.91599
[26]	Test-auc:0.91664
[27]	Test-auc:0.91665
[28]	Test-auc:0.91709
[29]	Test-auc:0.91753
[30]	Test-auc:0.91759
[31]	Test-auc:0.91860
[32]	Test-auc:0.91889
[33]	Test-auc:0.91918
[34]	Test-auc:0.91940
[35]	Test-auc:0.91957
[36]	Test-auc:0.92032
[37]	Test-auc:0.92037
[38]	Test-auc:0.92051
[39]	Test-auc:0.92055
[40]	Test-auc:0.92063
[41]	Test-auc:0.92087
[42]	Test-auc:0.92103
[43]	Test-auc:0.92156
[44]	Test-auc:0.92180
[45]	Test-auc:0.9220

In [28]:
y_test

32561    0
32562    0
32563    1
32564    1
32565    0
        ..
48837    0
48838    0
48839    0
48840    0
48841    1
Name: income, Length: 16281, dtype: int64

In [54]:
y_pred = model.predict(test_dmatrix)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)

In [49]:
y_pred

array([0.00415711, 0.16926189, 0.4439232 , ..., 0.7428167 , 0.05055859,
       0.7242275 ], dtype=float32)

In [50]:
eval_results = model.eval_set(
            evals=[(test_dmatrix, "valid")],
            iteration=model.num_boosted_rounds() - 1,
        )
auc = round(float(eval_results.split("\t")[1].split(":")[1]), 4)

In [51]:
auc

0.9274

In [52]:
roc_auc_score(y_test, y_pred)

0.9274362305413004

In [59]:
print("Accuracy:", accuracy_score(y_test, y_pred_binary))
print("\nClassification Report:\n", classification_report(y_test, y_pred_binary, target_names=label_encoder.classes_))
print("ROC-AUC:", roc_auc_score(y_test, y_pred))
print("f1_score:", f1_score(y_test, y_pred_binary))

Accuracy: 0.8746391499293655

Classification Report:
               precision    recall  f1-score   support

       <=50K       0.89      0.95      0.92     12435
        >50K       0.79      0.64      0.71      3846

    accuracy                           0.87     16281
   macro avg       0.84      0.79      0.81     16281
weighted avg       0.87      0.87      0.87     16281

ROC-AUC: 0.9274362305413004
f1_score: 0.7071315827234897


In [22]:
# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Train the XGBoost model directly
model = XGBClassifier(eval_metric="logloss", n_jobs=-1)
model.fit(X_train_preprocessed, y_train)

# Predict and evaluate
y_pred = model.predict(X_test_preprocessed)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test_preprocessed)[:, 1]))


Accuracy: 0.8724894048277133

Classification Report:
               precision    recall  f1-score   support

       <=50K       0.90      0.94      0.92     12435
        >50K       0.77      0.66      0.71      3846

    accuracy                           0.87     16281
   macro avg       0.83      0.80      0.81     16281
weighted avg       0.87      0.87      0.87     16281

ROC-AUC: 0.9256168477539263


#### Hyperparameter tunning XGBoost

In [ ]:
# Define the parameter grid
param_grid = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [3, 5, 7, 10],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0],
    "gamma": [0, 0.1, 0.2, 0.5],
    "reg_alpha": [0, 0.01, 0.1, 1],
    "reg_lambda": [0.1, 1, 10]
}

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

class SklearnXGBClassifier(XGBClassifier):
    def __sklearn_tags__(self):
        return {}

# Create the model
xgb = SklearnXGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)

# Use RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=50,  # Number of random combinations to try
    scoring="accuracy",
    cv=3,  # 3-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1  # Use all processors
)


# Fit RandomizedSearchCV on training data
random_search.fit(X_train_preprocessed, y_train)

# Best parameters and model evaluation
print("Best Parameters:", random_search.best_params_)

# Make predictions
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test_preprocessed)

# Evaluate the performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
